# GOOGL STOCK PRICE PREDICTION USING STACKED LSTM

# Importing the Libraries

In [67]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

%matplotlib inline

# Constants

In [26]:
TRAINING_DATASET = "Google_Stock_Price/Google_Stock_Price_Train.csv"
TEST_DATASET = "Google_Stock_Price/Google_Stock_Price_Test.csv"

TIMESTEPS = 60
NO_OF_INDICATORS = 1

# Part 1 - Data Preprocessing

# Importing the training set

In [27]:
dataset_train = pd.read_csv(TRAINING_DATASET)
training_set = dataset_train.iloc[:, 1:2].values # getting only the open column

# Feature Scaling

In [28]:
from sklearn.preprocessing import MinMaxScaler

sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)

# Creating a data structure with 60 timesteps and 1 output

In [29]:
X_train = []
y_train = []

training_set_size = training_set.shape[0]
for i in range(TIMESTEPS, training_set_size):
    X_train.append(training_set_scaled[i-TIMESTEPS:i, 0])
    y_train.append(training_set_scaled[i, 0])

X_train, y_train = np.array(X_train), np.array(y_train)

# Reshaping

In [30]:
X_train = np.reshape(a = X_train,
                    newshape = (X_train.shape[0], TIMESTEPS, NO_OF_INDICATORS))

# Part 2 - Building the RNN

# Initializing the RNN

In [39]:
regressor = Sequential()

# Adding the first LSTM layer and some Dropout regularization 

In [40]:
regressor.add(LSTM(units = 50,
                   return_sequences = True,
                   input_shape = (TIMESTEPS, NO_OF_INDICATORS)))
regressor.add(Dropout(0.2)) # 20% of the neurons will be droped-out during each iteration of the training

# Adding the second LSTM layer and some Dropout regularization 

In [41]:
regressor.add(LSTM(units = 50,
                   return_sequences = True))
regressor.add(Dropout(0.2)) # 20% of the neurons will be droped-out during each iteration of the training

# Adding the third LSTM layer and some Dropout regularization 

In [42]:
regressor.add(LSTM(units = 50,
                   return_sequences = True))
regressor.add(Dropout(0.2)) # 20% of the neurons will be droped-out during each iteration of the training

# Adding the fourth LSTM layer and some Dropout regularization 

In [43]:
regressor.add(LSTM(units = 50,
                   return_sequences = False)) # last LSTM layer and not going to return anymore sequence
regressor.add(Dropout(0.2)) # 20% of the neurons will be droped-out during each iteration of the training

# Adding the output layer

In [44]:
regressor.add(Dense(units = 1))

# Compiling the RNN

In [45]:
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error') # also RMSprop optimizer could be used

# Fitting the RNN to the training set

In [46]:
regressor.fit(X_train, y_train, epochs = 100, batch_size = 32)

Epoch 1/100
1198/1198 [==============================] - 14s 11ms/step - loss: 0.0432
Epoch 2/100
1198/1198 [==============================] - 8s 7ms/step - loss: 0.0071
Epoch 3/100
1198/1198 [==============================] - 8s 7ms/step - loss: 0.0055
Epoch 4/100
1198/1198 [==============================] - 8s 6ms/step - loss: 0.0046
Epoch 5/100
1198/1198 [==============================] - 8s 7ms/step - loss: 0.0047
Epoch 6/100
1198/1198 [==============================] - 7s 6ms/step - loss: 0.0046
Epoch 7/100
1198/1198 [==============================] - 7s 6ms/step - loss: 0.0049
Epoch 8/100
1198/1198 [==============================] - 7s 6ms/step - loss: 0.0048
Epoch 9/100
1198/1198 [==============================] - 7s 6ms/step - loss: 0.0045
Epoch 10/100
1198/1198 [==============================] - 5s 4ms/step - loss: 0.0044
Epoch 11/100
1198/1198 [==============================] - 6s 5ms/step - loss: 0.0040
Epoch 12/100
1198/1198 [==============================] - 5s 4ms/step - 

# Part 3 - Making the predictions and visualizing the results

# Getting the real stock price of January 2017

In [49]:
dataset_test = pd.read_csv(TEST_DATASET)
real_stock_price = dataset_test.iloc[:, 1:2].values # getting only the open column

# Getting the predicted stock price of January 2017

In [64]:
dataset_total = pd.concat((dataset_train['Open'], dataset_test['Open']), axis = 0)
inputs = dataset_total[len(dataset_total) - len(dataset_test) - TIMESTEPS : ].values
inputs = inputs.reshape(-1, 1) # reshaping to (INPUT_LENGTH, NO_OF_INDICATORS) as we are getting values
                               # directly as 1D array and not with iloc
inputs = sc.transform(inputs) # scaling the inputs
X_test = []
input_size = inputs.shape[0]
for i in range(TIMESTEPS, input_size):
    X_test.append(inputs[i-TIMESTEPS:i, 0])
X_test = np.array(X_test)
X_test = np.reshape(a = X_test,
                    newshape = (X_test.shape[0], TIMESTEPS, NO_OF_INDICATORS)) # reshaping for neural networks to predict
predicted_stock_price = regressor.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price) # inverse scaling to get the actual predicted values

# Visualizing the results

In [129]:
plot_colors = ['#ebac23', '#b80058', '#008cf9', '#006e00', '#00bbad', '#d163e6', '#b24502', '#ff9287', '#5954d6', '#00c6f8']

plt.figure(figsize=(16, 9), dpi=900)
plt.plot(real_stock_price, color=plot_colors[0], label="Real Google Stock Price")
plt.scatter(x = [i for i in range(len(real_stock_price))], y = real_stock_price, color=plot_colors[0])
plt.plot(predicted_stock_price, color=plot_colors[1], label="Predicted Google Stock Price")
plt.scatter(x = [i for i in range(len(predicted_stock_price))], y = predicted_stock_price, color=plot_colors[1])
plt.title('Google Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Google Stock Price')
plt.grid()
plt.legend(loc='best')
plt.savefig('Google Stock Price Prediction', dpi=300);